<a href="https://colab.research.google.com/github/Ha-eunYu/OTTO/blob/main/otto_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
# !pip install jsonlines
# import jsonlines

In [ ]:
# #Install Polars with all optional dependencies.
# !pip install 'polars[all]'

In [1]:
#importing Libraries
# import polars as pl
from io import StringIO
import pandas as pd
import json
import os
import numpy as np
import csv
from datetime import datetime
import time
from datetime import timedelta

## Kaggle_data

In [ ]:
# # 내 계정 API Token(json) 다운 받아서 업로드
# !pip install kaggle
# from google.colab import files
# files.upload()

In [ ]:
# # Kaggle API를 사용하기 위해서 json파일 ~/.kaggle로 이동
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

# #Permission Warning 방지
# !chmod 600 ~/.kaggle/kaggle.json

# # kaggle 데이터셋 API 주소 복사
# !kaggle competitions download -c otto-recommender-system

In [ ]:
# !unzip /content/otto-recommender-system.zip

In [ ]:
# test_df.to_csv('/content/drive/My Drive/OTTO/test.csv',index = False)

###test_data, test_data_df

In [ ]:
# # jsonl dataframe으로 읽기
# with open('test.jsonl', encoding='utf-8') as file:
#   test = [json.loads(line) for line in file]
# test=pd.DataFrame(test)
# test.head()

In [ ]:
# # session별 events풀기
# test_df = pd.DataFrame()
# chunks = pd.read_json('test.jsonl', lines=True, chunksize=100_000)

# for chunk in chunks:
#     event_dict = {'session': [],'aid': [],'ts': [],'type': []}
#     for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
#         for event in events:
#             event_dict['session'].append(session)
#             event_dict['aid'].append(event['aid'])
#             event_dict['ts'].append(event['ts'])
#             event_dict['type'].append(event['type'])
#     chunk_session = pd.DataFrame(event_dict)
#     test_df = pd.concat([test_df, chunk_session])
            
# test_df = test_df.reset_index(drop=True)
# test_df

In [ ]:
# test_df.to_csv('/content/drive/My Drive/OTTO/test.csv',index = False)
# # !cp test.csv "drive/My Drive/OTTO/"
# # test.to_csv("test.csv", index = False)

In [ ]:
# sample_sub.to_csv('/content/drive/My Drive/OTTO/sample_submission.csv',index = False)

## gdrive data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip -qq "/content/drive/MyDrive/OTTO/train1.zip"

In [ ]:
# train = pd.read_csv('train1.csv')
# train.session.nunique()
# train.type.nunique()
# train.shape

In [ ]:
# train[["session", "ts"]].groupby("session").head()

In [ ]:
# %%time
# train_df = train
# train_df["minutes"] = train_df[["session", "ts"]].groupby("session").diff(-1)*(-1/1000/60)
# train_df

In [ ]:
# train_df.to_csv('/content/drive/My Drive/OTTO/train1_df.csv',index = False)
# train_df.to_csv("train1_df.csv", index = False)

In [3]:
train_df = pd.read_csv('/content/drive/My Drive/OTTO/train1_df.csv')
train_df.session.nunique()

163473

In [ ]:
# 불가능
# train = pd.read_csv('/content/drive/My Drive/OTTO/train.csv')
# train.session.nunique()

## Create best-(sold, clicked) item item's list.

In [ ]:
# temp = train_df.groupby(['type','aid'])['session'].agg('count').reset_index()
# temp.columns = ['type','aid','count']
# temp.to_csv('/content/drive/My Drive/OTTO/temp.csv')
# order_num_df = temp.loc[(temp['type'] == 'orders'), ]
# order_num_df = order_num_df.sort_values(['count'],ascending=False).reset_index()
# order_num_df.to_csv('/content/drive/My Drive/OTTO/order_num_df.csv')
# clicks_num_df = temp.loc[(temp['type'] == 'clicks'), ]
# clicks_num_df = clicks_num_df.sort_values(['count'],ascending=False).reset_index()
# clicks_num_df.to_csv('/content/drive/My Drive/OTTO/clicks_num_df.csv')
# cart_num_df = temp.loc[(temp['type'] == 'cart'), ]
# cart_num_df = order_num_df.sort_values(['count'],ascending=False).reset_index()
# cart_num_df.to_csv('/content/drive/My Drive/OTTO/cart_num_df.csv')

In [4]:
temp = pd.read_csv('/content/drive/My Drive/OTTO/temp.csv')
order_num_df = pd.read_csv('/content/drive/My Drive/OTTO/order_num_df.csv')
clicks_num_df = pd.read_csv('/content/drive/My Drive/OTTO/clicks_num_df.csv')
cart_num_df = pd.read_csv('/content/drive/My Drive/OTTO/cart_num_df.csv')

In [5]:
order_num_df.aid = ' ' + order_num_df.aid.astype('str')
clicks_num_df.aid = ' ' + clicks_num_df.aid.astype('str')
cart_num_df.aid = ' ' + cart_num_df.aid.astype('str')
best_sold_list = order_num_df[:100].aid.sum()
best_clicked_list = clicks_num_df[:50].aid.sum()
best_shopping_list = cart_num_df[:20].aid.sum()

##test dataset
- 오래 본 아이템 추천하기

In [6]:
test = pd.read_csv('/content/drive/MyDrive/OTTO/test.csv')

In [ ]:
# test.to_csv("test.csv", index = False)

In [ ]:
# 6928123 4
test.shape

(6928123, 4)

In [ ]:
test

,session,aid,ts,type
0,12899779,59625,1661724000278,clicks
1,12899780,1142000,1661724000378,clicks
2,12899780,582732,1661724058352,clicks
3,12899780,973453,1661724109199,clicks
4,12899780,736515,1661724136868,clicks
...,...,...,...,...
6928118,14571577,1141710,1662328774770,clicks
6928119,14571578,519105,1662328775009,clicks
6928120,14571579,739876,1662328775605,clicks
6928121,14571580,202353,1662328781067,clicks


In [ ]:
# %%time
# test_df = test
# test_df["minutes"] = test[["session", "ts"]].groupby("session").diff(-1)*(-1/1000/60)

In [ ]:
# test_df.to_csv('/content/drive/My Drive/OTTO/test_df.csv',index = False)

In [7]:
test_df=pd.read_csv('/content/drive/My Drive/OTTO/test_df.csv')
test_df.head()

,session,aid,ts,type,minutes
0,12899779,59625,1661724000278,clicks,NaN
1,12899780,1142000,1661724000378,clicks,0.966233
2,12899780,582732,1661724058352,clicks,0.847450
3,12899780,973453,1661724109199,clicks,0.461150
4,12899780,736515,1661724136868,clicks,0.306333


In [8]:
test_df = test_df.sort_values(['minutes'],ascending=False)
test_df.head()

,session,aid,ts,type,minutes
30239,12905082,959154,1661726942565,clicks,10015.907300
4632,12900142,258596,1661724195582,clicks,10011.848333
17853,12902541,1544625,1661725403034,clicks,10007.667133
28235,12904668,510044,1661726708488,clicks,10002.046133
15991,12902229,1046138,1661725117749,clicks,9978.838333


In [9]:
test_df_1 = test_df.copy()
test_df_1.aid = ' ' + test_df_1.aid.astype('str')


In [ ]:
a=test_df_1.iloc[10000][0]
test_df_1.loc[(test_df["session"] == a)]

,session,aid,ts,type,minutes
1967597,13343260,1668343,1661875427904,clicks,6003.457667
1967589,13343260,737238,1661874518237,clicks,9.384900
1967600,13343260,1179529,1662235777320,carts,7.969600
1967590,13343260,1463310,1661875081331,clicks,2.482767
1967592,13343260,1463310,1661875239548,clicks,1.745917
1967598,13343260,925315,1662235635364,clicks,1.536983
1967599,13343260,507783,1662235727583,clicks,0.828950
1967593,13343260,301583,1661875344303,clicks,0.654017
1967596,13343260,42857,1661875407254,clicks,0.344167
1967595,13343260,301583,1661875390006,carts,0.287467


In [10]:
test_action_df = test_df.copy()
# test_action_df = test_action_df.sort_values(['minutes'],ascending=False)
test_action_df.aid = ' ' + test_df.aid.astype('str')
test_action_df = test_action_df.groupby(['session','type'])['aid'].sum().reset_index()
test_action_df

,session,type,aid
0,12899779,clicks,59625
1,12899780,clicks,1142000 582732 973453 736515 1142000
2,12899781,carts,199008
3,12899781,clicks,199008 194067 199008 199008 199008 199008 573...
4,12899782,carts,1494780 834354 975116 127404 413962 595994 13...
...,...,...,...
1948868,14571577,clicks,1141710
1948869,14571578,clicks,519105
1948870,14571579,clicks,739876
1948871,14571580,clicks,202353


### type: clicks인 경우 
- 이후에 carts

In [11]:
# type: clicks인 경우 복사, type: carts로 변경
next_carts_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ])
next_carts_df['type'] = 'carts'
next_carts_df

,session,type,aid
0,12899779,carts,59625
1,12899780,carts,1142000 582732 973453 736515 1142000
3,12899781,carts,199008 194067 199008 199008 199008 199008 573...
5,12899782,carts,603159 779477 1299062 602722 413962 975116 16...
7,12899783,carts,607638 1729553 255297 300127 1754419 1216820 ...
...,...,...,...
1948868,14571577,carts,1141710
1948869,14571578,carts,519105
1948870,14571579,carts,739876
1948871,14571580,carts,202353


### type: carts인 경우
- 이전에 클릭 추천
- 이후에 주문 추천

In [12]:
# type: carts인 경우 복사, type: clicks으로 변경
before_carts_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'carts'), ])
before_carts_df['type'] = 'clicks'
before_carts_df

,session,type,aid
2,12899781,clicks,199008
4,12899782,clicks,1494780 834354 975116 127404 413962 595994 13...
10,12899786,clicks,955252
12,12899787,clicks,1682750 1682750 1682750
16,12899790,clicks,1830166 1219653
...,...,...,...
1948716,14571430,clicks,903014
1948730,14571443,clicks,942326
1948774,14571486,clicks,350578
1948788,14571499,clicks,1132907


In [13]:
# type: carts인 경우 복사, type: orders로 변경
next_orders_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'carts'), ])
next_orders_df['type'] = 'orders'
next_orders_df

,session,type,aid
2,12899781,orders,199008
4,12899782,orders,1494780 834354 975116 127404 413962 595994 13...
10,12899786,orders,955252
12,12899787,orders,1682750 1682750 1682750
16,12899790,orders,1830166 1219653
...,...,...,...
1948716,14571430,orders,903014
1948730,14571443,orders,942326
1948774,14571486,orders,350578
1948788,14571499,orders,1132907


In [14]:
next_clicks_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ]).copy()
next_clicks_df

,session,type,aid
0,12899779,clicks,59625
1,12899780,clicks,1142000 582732 973453 736515 1142000
3,12899781,clicks,199008 194067 199008 199008 199008 199008 573...
5,12899782,clicks,603159 779477 1299062 602722 413962 975116 16...
7,12899783,clicks,607638 1729553 255297 300127 1754419 1216820 ...
...,...,...,...
1948868,14571577,clicks,1141710
1948869,14571578,clicks,519105
1948870,14571579,clicks,739876
1948871,14571580,clicks,202353


## 생각해보기

In [15]:
click_cart=set(next_clicks_df['session'].unique()) - set(next_orders_df['session'].unique())
print(len(test_action_df['session'].unique()), len(click_cart), len(test_action_df['session'].unique())-len(click_cart))

1671803 1428500 243303


In [16]:
# order: type이 crat인 aid와 click인 aid 병합
next_orders_df = pd.merge(next_orders_df, next_clicks_df[['session', 'aid']], on ='session', how = 'left')
next_orders_df["aid"] = next_orders_df["aid_x"] + next_orders_df["aid_y"]
next_orders_df = next_orders_df.drop(['aid_x', 'aid_y'], axis =1)
next_orders_df

,session,type,aid
0,12899781,orders,199008 199008 194067 199008 199008 199008 199...
1,12899782,orders,1494780 834354 975116 127404 413962 595994 13...
2,12899786,orders,955252 955252
3,12899787,orders,1682750 1682750 1682750 1682750 1024433
4,12899790,orders,1830166 1219653 1830166
...,...,...,...
242828,14571430,orders,903014 903014 1162324
242829,14571443,orders,942326 1407032 942326 568535
242830,14571486,orders,350578 350578 350578
242831,14571499,orders,1132907 1132907


In [ ]:
# next_crat: type이 click인 aid 병합

In [18]:
recommend_df = pd.concat([next_orders_df, next_carts_df, next_clicks_df], axis =0)
recommend_df["session_type"] = recommend_df["session"].astype('str') + "_" + recommend_df["type"] 
recommend_df

,session,type,aid,session_type
0,12899781,orders,199008 199008 194067 199008 199008 199008 199...,12899781_orders
1,12899782,orders,1494780 834354 975116 127404 413962 595994 13...,12899782_orders
2,12899786,orders,955252 955252,12899786_orders
3,12899787,orders,1682750 1682750 1682750 1682750 1024433,12899787_orders
4,12899790,orders,1830166 1219653 1830166,12899790_orders
...,...,...,...,...
1948868,14571577,clicks,1141710,14571577_clicks
1948869,14571578,clicks,519105,14571578_clicks
1948870,14571579,clicks,739876,14571579_clicks
1948871,14571580,clicks,202353,14571580_clicks


In [17]:
sample_sub = pd.read_csv('/content/drive/My Drive/OTTO/sample_submission.csv')
sample_sub

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524
...,...,...
5015404,14571580_carts,129004 126836 118524
5015405,14571580_orders,129004 126836 118524
5015406,14571581_clicks,129004 126836 118524
5015407,14571581_carts,129004 126836 118524


In [19]:
recommend_df[["session_type","aid"]]

,session_type,aid
0,12899781_orders,199008 199008 194067 199008 199008 199008 199...
1,12899782_orders,1494780 834354 975116 127404 413962 595994 13...
2,12899786_orders,955252 955252
3,12899787_orders,1682750 1682750 1682750 1682750 1024433
4,12899790_orders,1830166 1219653 1830166
...,...,...
1948868,14571577_clicks,1141710
1948869,14571578_clicks,519105
1948870,14571579_clicks,739876
1948871,14571580_clicks,202353


In [20]:
sample_sub_m = pd.merge(sample_sub, recommend_df[["session_type","aid"]], on = "session_type", how ="left")
sample_sub_m['next'] = sample_sub_m['aid'] + best_sold_list
sample_sub_m['next'].fillna(best_sold_list, inplace = True)
sample_sub_m['next'] = sample_sub_m['next'].str.strip()
sample_sub_m = sample_sub_m.drop(["labels", "aid"], axis = 1)
sample_sub_m.columns = ("session_type", "labels")
sample_sub_m

,session_type,labels
0,12899779_clicks,59625 80222 166037 231487 1733943 1022566 1629...
1,12899779_carts,59625 80222 166037 231487 1733943 1022566 1629...
2,12899779_orders,80222 166037 231487 1733943 1022566 1629608 14...
3,12899780_clicks,1142000 582732 973453 736515 1142000 80222 166...
4,12899780_carts,1142000 582732 973453 736515 1142000 80222 166...
...,...,...
5015404,14571580_carts,202353 80222 166037 231487 1733943 1022566 162...
5015405,14571580_orders,80222 166037 231487 1733943 1022566 1629608 14...
5015406,14571581_clicks,1100210 80222 166037 231487 1733943 1022566 16...
5015407,14571581_carts,1100210 80222 166037 231487 1733943 1022566 16...


In [21]:
sample_sub_m.to_csv('submission.csv', index=False)

In [ ]:
sample_sub_m.to_csv('/content/drive/My Drive/OTTO/submission.csv', index=False)

In [ ]:
kaggle competitions submit -c otto-recommender-system -f submission.csv -m "Message"